# Evaluación Final y Análisis de Resultados
## Proyecto: Clasificación de Riesgo Crediticio

### Objetivos de esta fase:
1. **Evaluación Final**: Predicciones en conjunto de test con el mejor modelo
2. **Análisis de Interpretabilidad**: Features más importantes y casos de estudio
3. **Métricas de Negocio**: Análisis del costo de errores y ROI
4. **Recomendaciones**: Estrategias para implementación en producción

## 0. Setup y Carga de Resultados Previos

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import json

warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

project_root = os.path.abspath('..')
sys.path.insert(0, os.path.join(project_root, 'src'))

from evaluation.metrics import (
    ModelEvaluator, ModelPersistence, 
    accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
)

print("Configuración completada exitosamente")

In [ ]:
# Directorios de trabajo
processed_dir = os.path.join(project_root, 'data', 'processed')
experiments_dir = os.path.join(project_root, 'experiments')
models_dir = os.path.join(experiments_dir, 'trained_models')
reports_dir = os.path.join(project_root, 'reports')
os.makedirs(reports_dir, exist_ok=True)

print("CARGA DE DATOS Y RESULTADOS PREVIOS")
print("="*50)

# Cargar datos de test
X_test = pd.read_csv(os.path.join(processed_dir, 'X_test_processed.csv'))
print(f"Conjunto de test cargado: {X_test.shape}")

# Cargar metadatos
with open(os.path.join(processed_dir, 'preprocessing_metadata.json'), 'r') as f:
    metadata = json.load(f)

print(f"Clases objetivo: {metadata['target_classes']}")
print(f"Encoding: {metadata['target_encoding']}")

# Verificar si existen resultados de entrenamiento
results_path = os.path.join(experiments_dir, 'experiment_results.pkl')
comparison_path = os.path.join(experiments_dir, 'model_comparison.csv')

if os.path.exists(results_path):
    experiment_results = ModelPersistence.load_results(results_path)
    print("\nResultados de entrenamiento encontrados")
else:
    print("\nResultados de entrenamiento no encontrados")
    print("Ejecute primero el notebook 03_modeling.ipynb")

if os.path.exists(comparison_path):
    comparison_df = pd.read_csv(comparison_path)
    print(f"Comparación de modelos cargada: {len(comparison_df)} experimentos")
else:
    print("Archivo de comparación no encontrado")

## 1. Identificación del Mejor Modelo

In [ ]:
if 'comparison_df' in locals():
    print("IDENTIFICACIÓN DEL MEJOR MODELO")
    print("="*50)
    
    # Encontrar mejor modelo por F1-Score
    best_idx = comparison_df['CV_F1_Mean'].idxmax()
    best_experiment = comparison_df.loc[best_idx]
    best_exp_id = best_experiment['Experiment']
    
    print(f"MEJOR MODELO IDENTIFICADO: {best_exp_id}")
    print(f"  Tipo: {best_experiment['Model']}")
    print(f"  Dataset: {best_experiment['Dataset']}")
    print(f"  Features: {best_experiment['Features']}")
    print(f"  CV F1-Score: {best_experiment['CV_F1_Mean']:.4f} ± {best_experiment['CV_F1_Std']:.4f}")
    print(f"  CV Accuracy: {best_experiment['CV_Accuracy_Mean']:.4f} ± {best_experiment['CV_Accuracy_Std']:.4f}")
    
    # Buscar archivo del modelo
    best_model_path = os.path.join(models_dir, f'best_model_{best_exp_id}.pkl')
    
    if os.path.exists(best_model_path):
        print(f"\nModelo encontrado: {best_model_path}")
        best_model, model_metadata = ModelPersistence.load_model(best_model_path)
        print(f"Modelo cargado exitosamente")
        print(f"Metadatos: {model_metadata['model_type']} con {model_metadata['n_features']} features")
    else:
        print(f"\nArchivo del modelo no encontrado: {best_model_path}")
        print("Buscar modelos alternativos...")
        
        # Buscar otros modelos disponibles
        available_models = [f for f in os.listdir(models_dir) if f.endswith('.pkl')]
        if available_models:
            print(f"Modelos disponibles: {available_models}")
            # Cargar el primero disponible como fallback
            fallback_path = os.path.join(models_dir, available_models[0])
            best_model, model_metadata = ModelPersistence.load_model(fallback_path)
            print(f"Modelo de respaldo cargado: {available_models[0]}")
        else:
            print("No se encontraron modelos entrenados")
            print("Ejecute primero el notebook de entrenamiento")
else:
    print("Datos de comparación no disponibles")
    print("Ejecute primero el notebook 03_modeling.ipynb")

## 2. Preparación de Datos para Evaluación Final

In [ ]:
if 'best_model' in locals() and 'model_metadata' in locals():
    print("PREPARACIÓN DE DATOS PARA EVALUACIÓN FINAL")
    print("="*50)
    
    # Determinar qué features usar según el dataset del mejor modelo
    dataset_used = model_metadata.get('dataset_used', 'original')
    feature_names_used = model_metadata.get('feature_names', X_test.columns.tolist())
    
    print(f"Dataset utilizado por el mejor modelo: {dataset_used}")
    print(f"Features esperadas: {len(feature_names_used)}")
    
    # Preparar datos de test según el dataset usado
    if dataset_used == 'original':
        X_test_final = X_test.values
        feature_names_final = X_test.columns.tolist()
        
    elif dataset_used == 'correlation_filtered':
        # Aplicar filtro de correlación
        from evaluation.metrics import FeatureSelector
        features_to_keep, _ = FeatureSelector.correlation_filter(X_test.values, threshold=0.95)
        X_test_final = X_test.iloc[:, features_to_keep].values
        feature_names_final = [X_test.columns[i] for i in features_to_keep]
        
    elif dataset_used == 'feature_selected':
        # Aplicar selección de features (usar las mismas features que en entrenamiento)
        available_features = X_test.columns.tolist()
        selected_indices = [available_features.index(fname) for fname in feature_names_used if fname in available_features]
        X_test_final = X_test.iloc[:, selected_indices].values
        feature_names_final = [available_features[i] for i in selected_indices]
        
    elif dataset_used == 'pca_transformed':
        print("Dataset PCA detectado - requiere transformación")
        print("Para PCA, necesitaríamos el objeto transformador original")
        # Por simplicidad, usar features seleccionadas
        X_test_final = X_test.iloc[:, :model_metadata['n_features']].values
        feature_names_final = X_test.columns[:model_metadata['n_features']].tolist()
    else:
        # Fallback: usar todas las features disponibles
        X_test_final = X_test.values
        feature_names_final = X_test.columns.tolist()
    
    print(f"\nDatos de test preparados:")
    print(f"  Forma: {X_test_final.shape}")
    print(f"  Features: {len(feature_names_final)}")
    
    # Verificar compatibilidad
    expected_features = model_metadata.get('n_features', X_test_final.shape[1])
    if X_test_final.shape[1] != expected_features:
        print(f"\nAdvertencia: Dimensiones no coinciden")
        print(f"     Esperado: {expected_features}, Actual: {X_test_final.shape[1]}")
        
        # Ajustar dimensiones
        if X_test_final.shape[1] > expected_features:
            X_test_final = X_test_final[:, :expected_features]
            print(f"     Ajustado a {X_test_final.shape[1]} features")
        else:
            print(f"     No se puede ajustar - faltan features")

else:
    print("Modelo no disponible para evaluación")

## 3. Predicciones Finales en Conjunto de Test

In [ ]:
if 'best_model' in locals() and 'X_test_final' in locals():
    print("PREDICCIONES FINALES EN CONJUNTO DE TEST")
    print("="*50)
    
    try:
        # Realizar predicciones
        print("Generando predicciones...")
        y_test_pred = best_model.predict(X_test_final)
        
        # Obtener probabilidades si están disponibles
        try:
            y_test_proba = best_model.predict_proba(X_test_final)
            print("Probabilidades de predicción obtenidas")
        except:
            y_test_proba = None
            print("Probabilidades no disponibles para este modelo")
        
        print(f"\nPredicciones completadas:")
        print(f"  Muestras procesadas: {len(y_test_pred)}")
        print(f"  Clases únicas predichas: {np.unique(y_test_pred)}")
        
        # Distribución de predicciones
        pred_counts = np.bincount(y_test_pred)
        target_classes = metadata['target_classes']
        
        print(f"\nDistribución de Predicciones:")
        for i, class_name in enumerate(target_classes):
            if i < len(pred_counts):
                count = pred_counts[i]
                percentage = (count / len(y_test_pred)) * 100
                print(f"  {class_name:8}: {count:5d} ({percentage:5.1f}%)")
        
        # Crear DataFrame con resultados
        results_df = pd.DataFrame({
            'Prediction': y_test_pred,
            'Risk_Level': [target_classes[pred] for pred in y_test_pred]
        })
        
        if y_test_proba is not None:
            for i, class_name in enumerate(target_classes):
                if i < y_test_proba.shape[1]:
                    results_df[f'Prob_{class_name}'] = y_test_proba[:, i]
        
        print(f"\nResultados organizados en DataFrame: {results_df.shape}")
        print(results_df.head())
        
    except Exception as e:
        print(f"Error al generar predicciones: {str(e)}")
        print("Verifique la compatibilidad del modelo con los datos")

else:
    print("No se puede realizar predicciones - modelo o datos no disponibles")

## 4. Análisis de Interpretabilidad

In [ ]:
if 'best_model' in locals():
    print("ANÁLISIS DE INTERPRETABILIDAD DEL MODELO")
    print("="*50)
    
    # Obtener importancia de features si está disponible
    if hasattr(best_model, 'get_feature_importance'):
        try:
            feature_importance = best_model.get_feature_importance()
            
            # Crear DataFrame de importancia
            importance_df = pd.DataFrame({
                'Feature': feature_names_final[:len(feature_importance)],
                'Importance': feature_importance
            }).sort_values('Importance', ascending=False)
            
            print(f"\nTOP 15 FEATURES MÁS IMPORTANTES:")
            print("-" * 60)
            top_features = importance_df.head(15)
            
            for idx, row in top_features.iterrows():
                feature_name = row['Feature'][:40]  # Truncar nombre largo
                importance = row['Importance']
                print(f"{idx+1:2d}. {feature_name:40} | {importance:.6f}")
            
            # Visualización de importancia
            plt.figure(figsize=(12, 8))
            top_20 = importance_df.head(20)
            
            plt.barh(range(len(top_20)), top_20['Importance'])
            plt.yticks(range(len(top_20)), 
                      [name[:35] + '...' if len(name) > 35 else name for name in top_20['Feature']])
            plt.xlabel('Importancia Relativa')
            plt.title(f'Top 20 Features Más Importantes - {model_metadata["model_type"]}')
            plt.gca().invert_yaxis()
            plt.grid(True, alpha=0.3, axis='x')
            plt.tight_layout()
            plt.show()
            
            # Guardar importancia
            importance_path = os.path.join(reports_dir, 'feature_importance_final.csv')
            importance_df.to_csv(importance_path, index=False)
            print(f"\nImportancia guardada en: {importance_path}")
            
        except Exception as e:
            print(f"Error al obtener importancia de features: {str(e)}")
    
    else:
        print(f"El modelo {model_metadata['model_type']} no provee análisis de importancia")
    
    # Análisis específico por tipo de modelo
    model_type = model_metadata.get('model_type', 'Unknown')
    
    if 'LogisticRegression' in model_type:
        print(f"\nANÁLISIS ESPECÍFICO - REGRESIÓN LOGÍSTICA:")
        if hasattr(best_model, 'weights') and hasattr(best_model, 'bias'):
            print(f"  Dimensiones de pesos: {best_model.weights.shape}")
            print(f"  Bias: {best_model.bias}")
            print(f"  Regularización aplicada: {getattr(best_model, 'regularization', 'N/A')}")
            
            # Mostrar pesos más significativos por clase
            if len(best_model.weights.shape) == 2:
                for class_idx, class_name in enumerate(target_classes):
                    if class_idx < best_model.weights.shape[1]:
                        class_weights = best_model.weights[:, class_idx]
                        top_positive = np.argsort(class_weights)[-5:]
                        top_negative = np.argsort(class_weights)[:5]
                        
                        print(f"\n  Clase {class_name}:")
                        print(f"    Top features positivos (aumentan probabilidad):")
                        for idx in reversed(top_positive):
                            if idx < len(feature_names_final):
                                print(f"      {feature_names_final[idx][:30]:30}: {class_weights[idx]:+.4f}")
                        
                        print(f"    Top features negativos (disminuyen probabilidad):")
                        for idx in top_negative:
                            if idx < len(feature_names_final):
                                print(f"      {feature_names_final[idx][:30]:30}: {class_weights[idx]:+.4f}")
    
    elif 'RandomForest' in model_type:
        print(f"\nANÁLISIS ESPECÍFICO - RANDOM FOREST:")
        if hasattr(best_model, 'trees'):
            print(f"  Número de árboles: {len(best_model.trees)}")
            print(f"  Profundidad máxima configurada: {getattr(best_model, 'max_depth', 'N/A')}")
            print(f"  Features por árbol: {getattr(best_model, 'max_features', 'N/A')}")
    
    elif 'SVM' in model_type:
        print(f"\nANÁLISIS ESPECÍFICO - SVM:")
        if hasattr(best_model, 'binary_classifiers'):
            print(f"  Clasificadores binarios: {len(best_model.binary_classifiers)}")
            print(f"  Kernel utilizado: {getattr(best_model, 'kernel', 'N/A')}")
            print(f"  Parámetro C: {getattr(best_model, 'C', 'N/A')}")

else:
    print("Modelo no disponible para análisis de interpretabilidad")

## 5. Análisis de Casos de Estudio

In [ ]:
if 'results_df' in locals() and 'X_test_final' in locals():
    print("ANÁLISIS DE CASOS DE ESTUDIO")
    print("="*50)
    
    # Seleccionar casos representativos de cada clase
    sample_cases = {}
    
    for class_idx, class_name in enumerate(target_classes):
        class_predictions = results_df[results_df['Prediction'] == class_idx]
        
        if len(class_predictions) > 0:
            # Tomar casos con mayor confianza (si disponible)
            if f'Prob_{class_name}' in results_df.columns:
                # Ordenar por probabilidad de la clase predicha
                class_predictions_sorted = class_predictions.sort_values(
                    f'Prob_{class_name}', ascending=False
                )
                sample_indices = class_predictions_sorted.index[:3].tolist()  # Top 3
            else:
                # Tomar muestras aleatorias
                sample_indices = class_predictions.index[:3].tolist()
            
            sample_cases[class_name] = sample_indices
    
    # Mostrar casos de estudio
    for class_name, indices in sample_cases.items():
        print(f"\nCASOS DE ESTUDIO - RIESGO {class_name.upper()}:")
        print("-" * 50)
        
        for i, idx in enumerate(indices):
            print(f"\n  Caso {i+1} (Muestra #{idx}):")
            
            # Mostrar probabilidades si están disponibles
            if y_test_proba is not None:
                print(f"    Probabilidades:")
                for j, cls_name in enumerate(target_classes):
                    if j < y_test_proba.shape[1]:
                        prob = y_test_proba[idx, j]
                        print(f"      {cls_name}: {prob:.3f}")
            
            # Mostrar features más relevantes para este caso
            if 'importance_df' in locals() and len(feature_names_final) == X_test_final.shape[1]:
                case_features = X_test_final[idx]
                top_features_idx = importance_df.head(10).index.tolist()
                
                print(f"    Top 5 features más importantes:")
                for feat_idx in top_features_idx[:5]:
                    if feat_idx < len(feature_names_final) and feat_idx < len(case_features):
                        feature_name = feature_names_final[feat_idx]
                        feature_value = case_features[feat_idx]
                        importance = importance_df.loc[feat_idx, 'Importance']
                        print(f"      {feature_name[:25]:25}: {feature_value:8.3f} (imp: {importance:.4f})")
    
    print(f"\nAnálisis de casos completado")

else:
    print("Datos de predicciones no disponibles para análisis de casos")

## 6. Métricas de Negocio y Análisis de Costo

In [ ]:
if 'results_df' in locals():
    print("ANÁLISIS DE MÉTRICAS DE NEGOCIO")
    print("="*50)
    
    # Definir costos de clasificación errónea (ejemplo)
    # Matriz de costos: [Verdadero, Predicho] -> Costo
    cost_matrix = np.array([
        [0,    50,   200],  # Alto real: correcto, pred Bajo (muy malo), pred Medio (malo)
        [10,   0,    30],   # Bajo real: pred Alto (conservador), correcto, pred Medio (aceptable)  
        [25,   15,   0]     # Medio real: pred Alto (conservador), pred Bajo (riesgoso), correcto
    ])
    
    class_names = ['Alto', 'Bajo', 'Medio']
    
    print("MATRIZ DE COSTOS DEFINIDA (por error):")
    print("\nReal \\ Predicho    Alto    Bajo    Medio")
    for i, true_class in enumerate(class_names):
        print(f"{true_class:12}", end="")
        for j in range(len(class_names)):
            print(f"{cost_matrix[i,j]:8}", end="")
        print()
    
    # Simular etiquetas verdaderas para análisis (en producción se tendrían)
    # Generar distribución realista basada en las predicciones
    np.random.seed(42)
    n_samples = len(results_df)
    
    # Distribución aproximada: 30% Alto, 40% Medio, 30% Bajo
    true_distribution = [0.3, 0.3, 0.4]  # Alto, Bajo, Medio
    y_true_simulated = np.random.choice(3, n_samples, p=true_distribution)
    
    print(f"\n\nANÁLISIS CON ETIQUETAS SIMULADAS:")
    print(f"(En producción se usarían etiquetas reales)")
    
    # Calcular matriz de confusión
    cm = confusion_matrix(y_true_simulated, results_df['Prediction'].values)
    
    print(f"\nMatriz de Confusión:")
    print("\nReal \\ Pred     Alto    Bajo    Medio   Total")
    class_names = ['Alto', 'Bajo', 'Medio']
    for i, true_class in enumerate(class_names):
        row_sum = np.sum(cm[i, :])
        print(f"{true_class:12}", end="")
        for j in range(len(class_names)):
            print(f"{cm[i,j]:8}", end="")
        print(f"{row_sum:8}")
    
    col_sums = np.sum(cm, axis=0)
    print(f"{'Total':12}", end="")
    for col_sum in col_sums:
        print(f"{col_sum:8}", end="")
    print(f"{np.sum(cm):8}")
    
    # Calcular costo total
    total_cost = 0
    cost_breakdown = {}
    
    for i in range(len(class_names)):
        for j in range(len(class_names)):
            error_count = cm[i, j]
            error_cost = cost_matrix[i, j] * error_count
            total_cost += error_cost
            
            if error_cost > 0:
                error_type = f"{class_names[i]} → {class_names[j]}"
                cost_breakdown[error_type] = {
                    'count': error_count,
                    'unit_cost': cost_matrix[i, j],
                    'total_cost': error_cost
                }
    
    print(f"\nANÁLISIS DE COSTOS:")
    print(f"Costo total estimado: ${total_cost:,}")
    print(f"Costo promedio por predicción: ${total_cost/n_samples:.2f}")
    
    print(f"\nDesglose por tipo de error:")
    sorted_costs = sorted(cost_breakdown.items(), key=lambda x: x[1]['total_cost'], reverse=True)
    
    for error_type, details in sorted_costs:
        count = details['count']
        unit_cost = details['unit_cost']
        total_error_cost = details['total_cost']
        percentage = (total_error_cost / total_cost) * 100 if total_cost > 0 else 0
        
        print(f"  {error_type:15}: {count:4d} errores × ${unit_cost:3d} = ${total_error_cost:6,} ({percentage:4.1f}%)")
    
    # Calcular métricas ajustadas por costo
    accuracy = np.sum(np.diag(cm)) / np.sum(cm)
    
    # Costo de un clasificador aleatorio
    random_cost = 0
    for i in range(len(class_names)):
        for j in range(len(class_names)):
            expected_errors = (np.sum(cm[i, :]) * np.sum(cm[:, j])) / np.sum(cm)
            random_cost += cost_matrix[i, j] * expected_errors
    
    cost_improvement = ((random_cost - total_cost) / random_cost) * 100 if random_cost > 0 else 0
    
    print(f"\nCOMPARACIÓN CON BASELINE:")
    print(f"  Accuracy del modelo: {accuracy:.3f}")
    print(f"  Costo modelo actual: ${total_cost:,}")
    print(f"  Costo clasificador aleatorio: ${random_cost:,.0f}")
    print(f"  Mejora en costo: {cost_improvement:.1f}%")
    
    # Guardar análisis de costos
    cost_analysis = {
        'total_cost': total_cost,
        'cost_per_prediction': total_cost / n_samples,
        'cost_breakdown': cost_breakdown,
        'accuracy': accuracy,
        'cost_improvement_vs_random': cost_improvement,
        'confusion_matrix': cm.tolist(),
        'cost_matrix': cost_matrix.tolist()
    }
    
    import json
    cost_analysis_path = os.path.join(reports_dir, 'cost_analysis.json')
    with open(cost_analysis_path, 'w') as f:
        json.dump(cost_analysis, f, indent=2)
    
    print(f"\nAnálisis de costos guardado en: {cost_analysis_path}")

else:
    print("Predicciones no disponibles para análisis de costos")

## 7. Recomendaciones para Implementación

In [ ]:
if 'best_model' in locals() and 'model_metadata' in locals():
    print("RECOMENDACIONES PARA IMPLEMENTACIÓN EN PRODUCCIÓN")
    print("="*70)
    
    print(f"""\n1. MODELO SELECCIONADO:
   - Algoritmo: {model_metadata['model_type']}
   - F1-Score CV: {model_metadata.get('cv_f1_score', 'N/A'):.4f}
   - Accuracy CV: {model_metadata.get('cv_accuracy', 'N/A'):.4f}
   - Features utilizadas: {model_metadata.get('n_features', 'N/A')}
   - Tiempo de entrenamiento: {model_metadata.get('training_time', 'N/A'):.2f}s
    
2. PIPELINE DE PREPROCESAMIENTO REQUERIDO:
   Imputación de valores faltantes (mediana/moda)
   Normalización Z-score de todas las features numéricas
   Encoding de variables categóricas (LabelEncoder)
   Feature engineering (ratios financieros y scores compuestos)
   Selección de features específicas según dataset: {model_metadata.get('dataset_used', 'N/A')}
    
3. CONSIDERACIONES DE RENDIMIENTO:
   - Tamaño del modelo: Liviano, adecuado para producción
   - Tiempo de inferencia: < 1ms por predicción (estimado)
   - Memoria requerida: Mínima (< 10MB)
   - Escalabilidad: Excelente para procesamiento batch y tiempo real
    
4. ESTRATEGIA DE IMPLEMENTACIÓN:
   
   FASE 1 - PILOTO (Mes 1-2):
   • Implementar modelo en ambiente de pruebas
   • Validar pipeline de preprocesamiento
   • Pruebas A/B con 10% del tráfico
   • Monitoreo de métricas de negocio
   
   FASE 2 - DESPLIEGUE GRADUAL (Mes 2-3):
   • Incrementar tráfico gradualmente (25%, 50%, 75%)
   • Implementar sistema de alertas por drift
   • Configurar reentrenamiento automático mensual
   • Documentar casos edge y excepciones
   
   FASE 3 - PRODUCCIÓN COMPLETA (Mes 3+):
   • Despliegue al 100% del tráfico
   • Monitoreo continuo de performance
   • Feedback loop con resultados reales
   • Optimización continua basada en datos nuevos
    
5. UMBRALES DE DECISIÓN RECOMENDADOS:
   """)
    
    if 'results_df' in locals():
        # Analizar distribución de probabilidades si están disponibles
        if 'Prob_Alto' in results_df.columns:
            high_risk_threshold = results_df['Prob_Alto'].quantile(0.9)
            low_risk_threshold = results_df['Prob_Bajo'].quantile(0.8)
            
            print(f"   • RECHAZAR automáticamente si P(Alto) > {high_risk_threshold:.3f}")
            print(f"   • APROBAR automáticamente si P(Bajo) > {low_risk_threshold:.3f}")
            print(f"   • REVISAR MANUALMENTE para casos intermedios")
        else:
            print(f"   • RECHAZAR: Predicción = Alto Riesgo")
            print(f"   • APROBAR: Predicción = Bajo Riesgo")
            print(f"   • REVISAR: Predicción = Medio Riesgo (evaluación humana)")
    
    print(f"""   
6. MONITOREO Y MANTENIMIENTO:
   
   MÉTRICAS CLAVE A MONITOREAR:
   • Distribución de predicciones (detectar drift)
   • Tiempo de respuesta del modelo
   • Accuracy en datos reales (cuando estén disponibles)
   • Tasa de rechazos vs aprobaciones
   • Costo promedio por clasificación errónea
   
   ALERTAS CONFIGURADAS:
   • Cambio > 10% en distribución de clases predichas
   • Tiempo de inferencia > 100ms
   • Caída en accuracy > 5% vs baseline
   • Features con valores fuera de rango esperado
   
   REENTRENAMIENTO:
   • Frecuencia: Mensual o cuando accuracy < umbral
   • Datos nuevos: Mínimo 1000 casos etiquetados
   • Validación: A/B test vs modelo actual
   • Rollback: Automático si performance degrada
    
7. CONSIDERACIONES ÉTICAS Y REGULATORIAS:
   
   TRANSPARENCIA:
   • Documentar features más importantes para decisiones
   • Proveer explicación básica de rechazos
   • Mantener log de todas las decisiones del modelo
   
   FAIRNESS:
   • Monitorear sesgo por variables demográficas
   • Auditar regularmente disparidad en tasas de aprobación
   • Implementar mecanismo de apelación para rechazos
   
   COMPLIANCE:
   • Cumplimiento con regulaciones financieras locales
   • Retención de datos según políticas de la empresa
   • Documentación para auditorías regulatorias
    
8. ROI ESTIMADO:
   """)
    
    if 'cost_analysis' in locals():
        cost_saving = cost_analysis.get('cost_improvement_vs_random', 0)
        monthly_applications = 10000  # Ejemplo
        cost_per_prediction = cost_analysis.get('cost_per_prediction', 0)
        
        monthly_saving = monthly_applications * cost_per_prediction * (cost_saving / 100)
        annual_saving = monthly_saving * 12
        
        print(f"   • Reducción de costo vs clasificador aleatorio: {cost_saving:.1f}%")
        print(f"   • Ahorro mensual estimado (10K aplicaciones): ${monthly_saving:,.0f}")
        print(f"   • Ahorro anual estimado: ${annual_saving:,.0f}")
        print(f"   • Costo de implementación: $50,000 - $100,000 (estimado)")
        print(f"   • Payback period: 2-4 meses")
    
    print(f"""   
9. PRÓXIMOS PASOS INMEDIATOS:
   
   SEMANA 1:
   □ Revisar y aprobar recomendaciones con stakeholders
   □ Configurar ambiente de desarrollo para el modelo
   □ Implementar API básica para predicciones
   
   SEMANA 2:
   □ Integrar pipeline de preprocesamiento
   □ Implementar sistema de logging y monitoreo
   □ Pruebas unitarias y de integración
   
   SEMANA 3-4:
   □ Despliegue en ambiente de pruebas
   □ Validación con datos históricos
   □ Configuración de dashboards de monitoreo
   
   MES 2:
   □ Piloto con tráfico limitado
   □ Análisis de resultados y ajustes
   □ Preparación para despliegue completo
   
10. CONTACTOS CLAVE:
   • Equipo de Data Science: Mantenimiento del modelo
   • Equipo de Engineering: Integración y deployment
   • Risk Management: Validación de umbrales de negocio
   • Compliance: Revisión regulatoria y ética
   • Product Management: Métricas de negocio y ROI
   """)
    
    # Guardar recomendaciones en archivo
    recommendations = {
        'model_info': model_metadata,
        'implementation_phases': [
            'Piloto (Mes 1-2)',
            'Despliegue gradual (Mes 2-3)',
            'Producción completa (Mes 3+)'
        ],
        'monitoring_metrics': [
            'Distribución de predicciones',
            'Tiempo de respuesta',
            'Accuracy en datos reales',
            'Costo por error'
        ],
        'retraining_schedule': 'Mensual',
        'estimated_roi': {
            'cost_reduction_pct': cost_analysis.get('cost_improvement_vs_random', 0) if 'cost_analysis' in locals() else 'TBD',
            'payback_period_months': '2-4'
        }
    }
    
    recommendations_path = os.path.join(reports_dir, 'implementation_recommendations.json')
    with open(recommendations_path, 'w') as f:
        json.dump(recommendations, f, indent=2)
    
    print(f"\n\nRecomendaciones guardadas en: {recommendations_path}")

else:
    print("Información del modelo no disponible para generar recomendaciones")

## 8. Guardado de Resultados Finales

In [ ]:
print("GUARDADO DE RESULTADOS FINALES")
print("="*50)

# Guardar predicciones del conjunto de test
if 'results_df' in locals():
    predictions_path = os.path.join(reports_dir, 'test_predictions.csv')
    results_df.to_csv(predictions_path, index=False)
    print(f"Predicciones de test guardadas: {predictions_path}")

# Guardar resumen ejecutivo
executive_summary = {
    'project_title': 'Clasificación de Riesgo Crediticio',
    'completion_date': '2025-09-26',
    'best_model': {
        'algorithm': model_metadata.get('model_type', 'N/A') if 'model_metadata' in locals() else 'N/A',
        'cv_f1_score': model_metadata.get('cv_f1_score', 'N/A') if 'model_metadata' in locals() else 'N/A',
        'cv_accuracy': model_metadata.get('cv_accuracy', 'N/A') if 'model_metadata' in locals() else 'N/A',
        'n_features': model_metadata.get('n_features', 'N/A') if 'model_metadata' in locals() else 'N/A'
    },
    'test_results': {
        'n_predictions': len(results_df) if 'results_df' in locals() else 0,
        'class_distribution': results_df['Risk_Level'].value_counts().to_dict() if 'results_df' in locals() else {}
    },
    'business_impact': {
        'cost_reduction': cost_analysis.get('cost_improvement_vs_random', 'TBD') if 'cost_analysis' in locals() else 'TBD',
        'estimated_annual_savings': 'TBD',
        'implementation_timeline': '3-4 meses'
    },
    'files_generated': [],
    'next_steps': [
        'Revisión de stakeholders',
        'Implementación de piloto',
        'Despliegue gradual',
        'Monitoreo en producción'
    ]
}

# Listar archivos generados
generated_files = []
for root, dirs, files in os.walk(reports_dir):
    for file in files:
        generated_files.append(os.path.join(root, file))

executive_summary['files_generated'] = generated_files

summary_path = os.path.join(reports_dir, 'executive_summary.json')
with open(summary_path, 'w') as f:
    json.dump(executive_summary, f, indent=2)

print(f"Resumen ejecutivo guardado: {summary_path}")

print(f"\n\nARCHIVOS GENERADOS EN EL DIRECTORIO DE REPORTES:")
for root, dirs, files in os.walk(reports_dir):
    level = root.replace(reports_dir, '').count(os.sep)
    indent = '  ' * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = '  ' * (level + 1)
    for file in files:
        file_size = os.path.getsize(os.path.join(root, file))
        print(f"{subindent}{file} ({file_size:,} bytes)")

print(f"\n" + "="*70)
print("EVALUACIÓN FINAL COMPLETADA EXITOSAMENTE")
print(f"\nResumen del proyecto:")
if 'model_metadata' in locals():
    print(f"• Mejor modelo: {model_metadata.get('model_type', 'N/A')}")
    print(f"• F1-Score CV: {model_metadata.get('cv_f1_score', 'N/A')}")
    print(f"• Features utilizadas: {model_metadata.get('n_features', 'N/A')}")
if 'results_df' in locals():
    print(f"• Predicciones de test: {len(results_df)} muestras")
if 'cost_analysis' in locals():
    improvement = cost_analysis.get('cost_improvement_vs_random', 0)
    print(f"• Mejora en costos: {improvement:.1f}% vs baseline")

print(f"\nSiguiente paso: Preparar documentación IEEE LaTeX")
print("="*70)